In [111]:
import requests
import json
import pandas as pd
from config import api_key
import warnings
warnings.filterwarnings('ignore')

In [84]:
#write api call for 2017 data and get response
long_url2 = "https://api.census.gov/data/2017/acs/acs5/?get=B01001_001E,B19013_001E,NAME&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key="
response2 = requests.get(long_url2+api_key)
print(response2)

<Response [200]>


In [85]:
#read json
response2=response2.json()

In [86]:
#convert to dataframe and clean up header
census_pd_2017 = pd.DataFrame(response2)
census_pd_2017.columns = census_pd_2017.iloc[0]
census_pd_2017 = census_pd_2017.iloc[1:]
census_pd_2017.head()


,B01001_001E,B19013_001E,NAME,metropolitan statistical area/micropolitan statistical area
1,42608,54533,"Aberdeen, SD Micro Area",10100
2,71454,45483,"Aberdeen, WA Micro Area",10140
3,169000,48156,"Abilene, TX Metro Area",10180
4,38289,46689,"Ada, OK Micro Area",10220
5,18525,11680,"Adjuntas, PR Micro Area",10260


In [87]:
#rename columns
census_pd_2017 = census_pd_2017.rename(columns={"B01001_001E": "2017 Pop","B19013_001E": "2017 Household Income","NAME": "Metro Area", "metropolitan statistical area/micropolitan statistical area":"Metro Code"})
census_pd_2017.head()

,2017 Pop,2017 Household Income,Metro Area,Metro Code
1,42608,54533,"Aberdeen, SD Micro Area",10100
2,71454,45483,"Aberdeen, WA Micro Area",10140
3,169000,48156,"Abilene, TX Metro Area",10180
4,38289,46689,"Ada, OK Micro Area",10220
5,18525,11680,"Adjuntas, PR Micro Area",10260


In [80]:
#make API call for 2012 data and read json
long_url3 = "https://api.census.gov/data/2012/acs/acs5/?get=B01001_001E,B19013_001E,NAME&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key="
response3 = requests.get(long_url3+api_key)
print(response3)

response3=response3.json()

<Response [200]>


In [88]:
#convert to dataframe
census_pd_2012 = pd.DataFrame(response3)
census_pd_2012.columns = census_pd_2012.iloc[0]
census_pd_2012 = census_pd_2012.iloc[1:]
census_pd_2012.head()

,B01001_001E,B19013_001E,NAME,metropolitan statistical area/micropolitan statistical area
1,58041,44339,"Abbeville, LA Micro Area",10020
2,40612,49652,"Aberdeen, SD Micro Area",10100
3,72272,42440,"Aberdeen, WA Micro Area",10140
4,165083,44007,"Abilene, TX Metro Area",10180
5,37438,40680,"Ada, OK Micro Area",10220


In [89]:
#rename columns
census_pd_2012 = census_pd_2012.rename(columns={"B01001_001E": "2012 Pop","B19013_001E": "2012 Household Income","NAME": "Metro Area", "metropolitan statistical area/micropolitan statistical area":"Metro Code"})
census_pd_2012.head()

,2012 Pop,2012 Household Income,Metro Area,Metro Code
1,58041,44339,"Abbeville, LA Micro Area",10020
2,40612,49652,"Aberdeen, SD Micro Area",10100
3,72272,42440,"Aberdeen, WA Micro Area",10140
4,165083,44007,"Abilene, TX Metro Area",10180
5,37438,40680,"Ada, OK Micro Area",10220


In [90]:
#merge 2013 and 2017 df's 
census_final = pd.merge(census_pd_2012,census_pd_2017, on="Metro Area", how="inner")

In [91]:
census_final.head()

,2012 Pop,2012 Household Income,Metro Area,Metro Code_x,2017 Pop,2017 Household Income,Metro Code_y
0,40612,49652,"Aberdeen, SD Micro Area",10100,42608,54533,10100
1,72272,42440,"Aberdeen, WA Micro Area",10140,71454,45483,10140
2,165083,44007,"Abilene, TX Metro Area",10180,169000,48156,10180
3,37438,40680,"Ada, OK Micro Area",10220,38289,46689,10220
4,19458,13095,"Adjuntas, PR Micro Area",10260,18525,11680,10260


In [ ]:
#clean up columns
census_final = census_final.drop(census_final.columns[[3, 6]], axis=1)

In [94]:
census_final.head()

,2012 Pop,2012 Household Income,Metro Area,2017 Pop,2017 Household Income
0,40612,49652,"Aberdeen, SD Micro Area",42608,54533
1,72272,42440,"Aberdeen, WA Micro Area",71454,45483
2,165083,44007,"Abilene, TX Metro Area",169000,48156
3,37438,40680,"Ada, OK Micro Area",38289,46689
4,19458,13095,"Adjuntas, PR Micro Area",18525,11680


In [95]:
#reorder columns
census_final = census_final[['Metro Area','2012 Pop', '2012 Household Income','2017 Pop', '2017 Household Income']]

In [96]:
census_final.head()

,Metro Area,2012 Pop,2012 Household Income,2017 Pop,2017 Household Income
0,"Aberdeen, SD Micro Area",40612,49652,42608,54533
1,"Aberdeen, WA Micro Area",72272,42440,71454,45483
2,"Abilene, TX Metro Area",165083,44007,169000,48156
3,"Ada, OK Micro Area",37438,40680,38289,46689
4,"Adjuntas, PR Micro Area",19458,13095,18525,11680


In [97]:
#export to CSV 
census_final.to_csv("census.csv")

In [105]:
#census_final.loc[census_final['2017 Pop'] > 200000]
census_final.dtypes

0
Metro Area               object
2012 Pop                 object
2012 Household Income    object
2017 Pop                 object
2017 Household Income    object
dtype: object

In [130]:
census_final[["2012 Pop", "2012 Household Income","2017 Pop","2017 Household Income"]] = census_final[["2012 Pop", "2012 Household Income","2017 Pop","2017 Household Income"]].apply(pd.to_numeric)

In [107]:
census_final.dtypes

0
Metro Area               object
2012 Pop                  int64
2012 Household Income     int64
2017 Pop                  int64
2017 Household Income     int64
dtype: object

In [131]:
census_filter = census_final.loc[census_final['2017 Pop'] > 200000]
census_filter.head()

,Metro Area,2012 Pop,2012 Household Income,2017 Pop,2017 Household Income
6,"Akron, OH Metro Area",702966,49936,703398,53418
9,"Albany-Schenectady-Troy, NY Metro Area",870890,60841,881862,65743
13,"Albuquerque, NM Metro Area",885683,48990,905049,50781
18,"Allentown-Bethlehem-Easton, PA-NJ Metro Area",821273,58111,832790,62479
23,"Amarillo, TX Metro Area",250224,47866,261827,53665


In [132]:
census_filter["Population Growth"] = (census_filter["2017 Pop"]/census_filter["2012 Pop"]) - 1 
census_filter["Income Growth"] = (census_filter["2017 Household Income"]/census_filter["2012 Household Income"]) - 1 

In [114]:
census_filter.head()

,Metro Area,2012 Pop,2012 Household Income,2017 Pop,2017 Household Income,Population Growth,Income Growth
6,"Akron, OH Metro Area",702966,49936,703398,53418,0.000615,0.069729
9,"Albany-Schenectady-Troy, NY Metro Area",870890,60841,881862,65743,0.012599,0.080571
13,"Albuquerque, NM Metro Area",885683,48990,905049,50781,0.021866,0.036558
18,"Allentown-Bethlehem-Easton, PA-NJ Metro Area",821273,58111,832790,62479,0.014023,0.075166
23,"Amarillo, TX Metro Area",250224,47866,261827,53665,0.046370,0.121151


In [133]:
census_filter["Population Rank"] = census_filter["Population Growth"].rank() 

In [134]:
census_filter["Income Rank"] = census_filter["Income Growth"].rank() 

In [135]:
census_filter["Total Score"] = census_filter["Population Rank"] + census_filter["Income Rank"]

In [141]:
census_filter
census_filter.sort_values(["Total Score"], ascending=[False] )
reset_index(drop=True,inplace=True)

,Metro Area,2012 Pop,2012 Household Income,2017 Pop,2017 Household Income,Population Growth,Income Growth,Population Rank,Income Rank,Total Score
63,"Grand Rapids-Wyoming, MI Metro Area",777159,50491,1039182,58094,0.337155,0.150581,163.0,161.0,324.0
105,"Huntington-Ashland, WV-KY-OH Metro Area",287234,38797,360603,44315,0.255433,0.142227,160.0,158.0,318.0
67,"Greeley, CO Metro Area",253552,56589,285729,66489,0.126905,0.174946,153.0,165.0,318.0
588,"Nashville-Davidson--Murfreesboro--Franklin, TN...",1595454,52779,1830410,59365,0.147266,0.124784,156.0,153.0,309.0
465,"Fargo, ND-MN Metro Area",209177,51411,232660,59074,0.112264,0.149054,147.0,160.0,307.0
515,"Tuscaloosa, AL Metro Area",219071,41557,239589,48344,0.093659,0.163318,141.0,163.0,304.0
379,"College Station-Bryan, TX Metro Area",227843,38599,248554,45078,0.090900,0.167854,139.0,164.0,303.0
470,"Fayetteville-Springdale-Rogers, AR-MO Metro Area",464729,46428,514166,52551,0.106378,0.131882,144.0,156.0,300.0
672,"Provo-Orem, UT Metro Area",526804,59594,587190,66742,0.114627,0.119945,149.0,146.0,295.0
731,"Seattle-Tacoma-Bellevue, WA Metro Area",3453748,67437,3735216,77269,0.081496,0.145795,133.0,159.0,292.0
